# Training CommonandLineTool

This notebook provide an step-by-step instruction to explain how to wrap the `training` step as a Common Workflow Language CommandLineTool and execute it with a CWL runner.

> Note: Before proceeding, make sure to select the correct kernel. In the top-right corner of the notebook, choose the Jupyter kernel named `Bash`.



## Setup

In [1]:
export WORKSPACE=/workspace/machine-learning-process
export RUNTIME=${WORKSPACE}/runs
mkdir -p ${RUNTIME}
cd ${RUNTIME}
printenv | grep RUNTIME
pwd

XDG_RUNTIME_DIR=/workspace/.local
RUNTIME=/workspace/machine-learning-process/runs
/workspace/machine-learning-process/runs


## Run the training

Inspect and use `cwltool` to run the CommandLineTool definition:

The CWL document below shows the `crop` step wrapped as a CWL CommandLineTool:

In [17]:
cat ${WORKSPACE}/training/app-package/tile-sat-training.cwl | yq 



{
  "cwlVersion": "v1.2",
  "$namespaces": {
    "s": "https://schema.org/"
  },
  "s:softwareVersion": "1.0.8",
  "schemas": [
    "http://schema.org/version/9.0/schemaorg-current-http.rdf"
  ],
  "$graph": [
    {
      "class": "Workflow",
      "id": "tile-sat-training",
      "label": "Tile-based calssifier on EuroSAT data",
      "doc": "Training a CNN on Sentinel-2 data(EuroSAT) to classify small patch of image into different landcover classes.",
      "requirements": [
        {
          "class": "InlineJavascriptRequirement"
        },
        {
          "class": "ScatterFeatureRequirement"
        }
      ],
      "inputs": {
        "MLFLOW_TRACKING_URI": {
          "label": "MLFLOW_TRACKING_URI",
          "type": "string"
        },
        "stac_reference": {
          "label": "stac_reference",
          "doc": "STAC Item label url",
          "type": "string"
        },
        "BATCH_SIZE": {
          "label": "BATCH_SIZE",
          "default": 4,
          "doc": 

Inspect the docker refrence


In [21]:
yq '.["$graph"][] | select(.class == "CommandLineTool") | .hints.DockerRequirement.dockerPull' ${WORKSPACE}/training/app-package/tile-sat-training.cwl


"train:latest"


Updating the docker refrence with the latest verion

In [22]:
VERSION=$(curl -s https://api.github.com/repos/eoap/machine-learning-process/releases/latest | jq -r '.tag_name')
curl -L -o ${WORKSPACE}/training/app-package/tile-sat-training.cwl \
  "https://github.com/eoap/machine-learning-process/releases/download/${VERSION}/tile-sat-training.${VERSION}.cwl"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  5546  100  5546    0     0   5879      0 --:--:-- --:--:-- --:--:-- 1387k


In [23]:
yq '.["$graph"][] | select(.class == "CommandLineTool") | .hints.DockerRequirement.dockerPull' ${WORKSPACE}/training/app-package/tile-sat-training.cwl


"ghcr.io/eoap/machine-learning-process/training@sha256:cbb97e479c9c5ca3b15257d034b0fce4ac5cba4e60e4b128b0fbe18f657a743f"


Run the CWL description for the `training` command line tool:

In [33]:
cwltool \
    --debug \
    --outdir ${WORKSPACE}/runs \
    ${WORKSPACE}/training/app-package/tile-sat-training.cwl#tile-sat-training \
    --MLFLOW_TRACKING_URI http://my-mlflow:5000 \
    --stac_reference https://raw.githubusercontent.com/eoap/machine-learning-process/main/training/app-package/EUROSAT-Training-Dataset/catalog.json \
    --BATCH_SIZE 2 \
    --CLASSES 10 \
    --DECAY 0.1 \
    --EPOCHS 5 \
    --EPSILON 0.000001 \
    --LEARNING_RATE 0.0001 \
    --LOSS categorical_crossentropy \
    --MEMENTUM 0.95 \
    --OPTIMIZER Adam \
    --REGULARIZER None \
    --SAMPLES_PER_CLASS 5

INFO /workspace/.venv/bin/cwltool 3.1.20240708091337
INFO Resolved '/workspace/machine-learning-process/training/app-package/tile-sat-training.cwl#tile-sat-training' to 'file:///workspace/machine-learning-process/training/app-package/tile-sat-training.cwl#tile-sat-training'
ERROR Unhandled error:
  'int' object has no attribute 'clear'
Traceback (most recent call last):
  File "/workspace/.venv/lib/python3.9/site-packages/cwltool/main.py", line 1275, in main
    initialized_job_order_object = init_job_order(
  File "/workspace/.venv/lib/python3.9/site-packages/cwltool/main.py", line 440, in init_job_order
    cmd_line = vars(toolparser.parse_args(args.job_order))
  File "/opt/conda/lib/python3.9/argparse.py", line 1825, in parse_args
    args, argv = self.parse_known_args(args, namespace)
  File "/opt/conda/lib/python3.9/argparse.py", line 1858, in parse_known_args
    namespace, args = self._parse_known_args(args, namespace)
  File "/opt/conda/lib/python3.9/argparse.py", line 2067, in

: 1

Let's look at the content of the stderr:

In [ ]:
cat crop.log

Remember the `cwltool` CWL runner can also take the parameters as a YAML file:


In [ ]:
cat << EOF > params.yaml
item: https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2B_10TFK_20210713_0_L2A
aoi: "-121.399,39.834,-120.74,40.472"
epsg: "EPSG:4326"
band: nir
EOF

Invoke `crop` against the NIR band redirecting the stdout to a file:

In [ ]:
cwltool \
    --podman \
    --outdir ${WORKSPACE}/runs \
    ${WORKSPACE}/cwl-cli/crop.cwl \
    params.yaml > results.json 

Let's inspect the stdout produced. The `output` block with the id `cropped` of the CWL description is a file:

In [ ]:
cat results.json

In [ ]:
cat ${WORKSPACE}/cwl-cli/crop.cwl | yq e .outputs -

Let's list the results produced:

In [ ]:
ls -l ${RUNTIME}